In [17]:
import respy as rp
import pandas as pd
import numpy as np

from auxiliary import *

pd.set_option('display.max_rows', 1000)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [34]:
params, options, data = rp.get_example_model("kw_97_basic")

num_occupations, n_periods = 3, 3

params_occ, options_occ = update_model_specification(params, options, num_occupations)

In [36]:
params_occ.loc["shocks_sdcorr"]

,value,sd,comment
name,,,
sd_aa,1.0,NaN,NaN
sd_bb,1.0,NaN,NaN
sd_blue_collar,1.0,NaN,NaN
sd_cc,1.0,NaN,NaN
sd_military,1.0,NaN,NaN
sd_white_collar,1.0,NaN,NaN
sd_school,1.0,NaN,NaN
sd_home,1.0,NaN,NaN
corr_bb_aa,0.0,NaN,NaN
